In [17]:
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [18]:
df = pd.read_csv("Data/Joules_of_Siam_Data - Dataset_Strech.csv")
df.head()

df = df[::-1]
df.head()

,Year,Month,Date,Population,Temperature,CPI,GDP,Peak
251,2002,1,01-01-02,63.249167,26.4525,70.233333,480343.3333,14552.5
250,2002,2,01-02-02,63.208333,26.4850,70.233333,480343.3333,15260.8
249,2002,3,01-03-02,63.167500,26.5175,70.233333,480343.3333,16485.3
248,2002,4,01-04-02,63.126667,26.5500,70.233333,470404.6667,16681.1
247,2002,5,01-05-02,63.085833,26.5825,70.066667,470404.6667,16293.1


In [19]:
# Convert Date data to Datetime

df['Date']= pd.to_datetime(df['Date'])
df['Year']= df['Date'].dt.year
df['Month']= df['Date'].dt.month

df.set_index("Date", inplace=True)


In [20]:
# Split train, test

train = df[df.Year < 2020]
test  = df[df.Year >= 2020]

print(len(train))
print(len(test))

216
36


In [6]:
import pmdarima as pm

model = pm.auto_arima(train['Peak'], 
                        m=12, seasonal=True,
                      start_p=0, start_q=0, max_order=4, test='adf',error_action='ignore',  
                           suppress_warnings=True,
                      stepwise=True, trace=True)

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(1,0,1)[12] intercept   : AIC=inf, Time=8.62 sec
 ARIMA(0,0,0)(0,0,0)[12] intercept   : AIC=4159.428, Time=0.09 sec
 ARIMA(1,0,0)(1,0,0)[12] intercept   : AIC=3463.791, Time=6.26 sec
 ARIMA(0,0,1)(0,0,1)[12] intercept   : AIC=3775.246, Time=10.74 sec
 ARIMA(0,0,0)(0,0,0)[12]             : AIC=4951.618, Time=0.07 sec
 ARIMA(1,0,0)(0,0,0)[12] intercept   : AIC=3527.255, Time=0.05 sec
 ARIMA(1,0,0)(2,0,0)[12] intercept   : AIC=3450.350, Time=16.15 sec
 ARIMA(1,0,0)(2,0,1)[12] intercept   : AIC=3437.903, Time=17.30 sec
 ARIMA(1,0,0)(1,0,1)[12] intercept   : AIC=3433.114, Time=10.99 sec
 ARIMA(1,0,0)(0,0,1)[12] intercept   : AIC=3487.562, Time=5.15 sec
 ARIMA(1,0,0)(1,0,2)[12] intercept   : AIC=3438.713, Time=19.68 sec
 ARIMA(1,0,0)(0,0,2)[12] intercept   : AIC=3480.272, Time=8.16 sec
 ARIMA(1,0,0)(2,0,2)[12] intercept   : AIC=inf, Time=19.71 sec
 ARIMA(2,0,0)(1,0,1)[12] intercept   : AIC=3448.758, Time=11.83 sec
 ARIMA(1,0,1)(1,0,1)[1

In [9]:
import pickle
file = open("./Models/ARIMA_model.pkl", "wb")
pickle.dump(obj=model, file=file)

In [10]:
file = open("./Models/ARIMA_model.pkl", "rb")
test = pickle.load(file)

In [22]:
model.fit(train['Peak'])

ARIMA(order=(3, 0, 0), scoring_args={}, seasonal_order=(1, 0, 1, 12),
      suppress_warnings=True, with_intercept=False)

In [37]:
results = model.predict(n_periods=120)

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [39]:
Arima_values = results.values

In [42]:
file = open("./Data/Dashboard_Data/forecast_ARIMA.pkl", "wb")
pickle.dump(Arima_values, file)

In [75]:
# Split train set and test set from train for training.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train.drop('Peak',axis=1), 
                                                    train['Peak'], test_size=0.30)

In [76]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 151 entries, 2011-01-07 to 2016-01-11
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Year         151 non-null    int64  
 1   Month        151 non-null    int64  
 2   Population   151 non-null    float64
 3   Temperature  151 non-null    float64
 4   CPI          151 non-null    float64
 5   GDP          151 non-null    float64
dtypes: float64(4), int64(2)
memory usage: 8.3 KB


In [77]:
import xgboost
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

model = XGBRegressor(objective='count:poisson',n_estimators=300, verbosity=3)

# Various hyper-parameters to tune
xgb1 = XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:squarederror', 'count:poisson'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [50, 100, 500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=False)


#xgbmodel = model.fit(X_train, y_train)

xgb_grid.fit(X_train, y_train)
print(xgb_grid.best_score_)
print(xgb_grid.best_params_) 

[18:24:11] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.
[18:24:11] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.


[18:24:11] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

[18:24:11] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

[18:24:11] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

[18:24:11] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

[18:24:11] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

[18:24:11] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

[18:24:12] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

[18:24:12] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

[18:24:12] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

[18:24:13] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not

In [78]:
# Train using the best parameters
#parameters = {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 7, 'min_child_weight': 4, 'n_estimators': 100, 'nthread': 4, 'objective': 'reg:squarederror', 'silent': 1, 'subsample': 0.7}
# For 4 features
#model = XGBRegressor(colsample_bytree=0.7, learning_rate=0.05, max_depth=7, min_child_weight=4, n_estimators=100, n_thread=4, objective='reg:squarederror', silent=1, subsample=0.7)

# For 5 features
model = XGBRegressor(colsample_bytree=0.7, learning_rate=0.05, max_depth=7, min_child_weight=4, n_estimators=500, n_thread=4, objective='count:poisson', silent=1, subsample=0.7)

In [79]:
# Fit model
xgbmodel = model.fit(X_train, y_train)

[18:26:46] WARNING: ../src/learner.cc:767: 
Parameters: { "n_thread", "silent" } are not used.



In [80]:
# Hmmm, a lot of errors

from sklearn.metrics import mean_squared_error
from math import sqrt

predictions_xgb = model.predict(X_test)
rmse_xgb = sqrt(mean_squared_error(y_test, predictions_xgb))
print("XGBoost - Root Mean Square Error (RMSE): %.3f" % rmse_xgb)

XGBoost - Root Mean Square Error (RMSE): 869.025


In [81]:
predictions_xgb

array([21807.918, 20343.31 , 26161.666, 23725.88 , 24503.568, 17492.85 ,
       20832.83 , 27288.691, 22748.21 , 20909.72 , 26498.238, 18378.562,
       16257.786, 27165.2  , 21492.844, 24348.592, 22291.484, 26075.084,
       21441.621, 21010.727, 17151.758, 20433.316, 26098.943, 26585.084,
       28137.56 , 26465.938, 18250.379, 21145.023, 18484.996, 16352.593,
       22575.727, 17151.758, 22291.484, 30228.393, 27205.436, 18253.756,
       15906.67 , 16357.038, 22227.863, 18407.785, 21044.234, 22302.266,
       18345.693, 25058.902, 24700.135, 23258.084, 23403.13 , 20041.178,
       21722.586, 18397.361, 19291.193, 24096.361, 22263.123, 20539.6  ,
       23702.045, 20854.695, 18457.656, 25087.453, 21035.146, 16560.508,
       19987.295, 26217.812, 25476.764, 23702.045, 19003.309],
      dtype=float32)

In [82]:
# Normalize root mean squared, pretty good

rmse_xgb/(train['Peak'].max()-train['Peak'].min())

0.053312136404359486

In [83]:
# Set index for test set
test = test.set_index('Date')

In [84]:
test_wo_peak = test.drop('Peak',axis=1)

In [85]:
# Make predictions

predictions = model.predict(test_wo_peak)
#predictions = pd.DataFrame({'Predictions': predictions})

In [86]:
# Test with unobserved data
predictions
rmse_xgb_test = sqrt(mean_squared_error(test["Peak"], predictions))
print("XGBoost - Root Mean Square Error (RMSE) for test set: %.3f" % rmse_xgb_test)

XGBoost - Root Mean Square Error (RMSE) for test set: 2168.758


In [87]:
# Normalize root mean squared, pretty good
rmse_xgb/(test['Peak'].max()-test['Peak'].min())

0.1305312938426074